# Esercizio 2 - Content2Form

In questo esercizio partendo dalle stesse definizioni date dagli studenti usate nell'esercizio 1, si vuole provare a inferire il miglior synset corrispondente a quelle definizioni.  
Si tratta di **Onomasiologic search**.

### Idea:
- Esattamente come nell'esercizio 1, si salva l'input in un dizionario e si pulisce il dizionario
- Si trovano le parole più frequenti per ogni lista di definizioni relative ad ogni termine (come nell'esercizio 1)
- Si considerano le parole più frequenti come **genus** 
- Si preleva da wordnet synset name, definition e examples per *ogni* **iponimo** di ogni **genus**
- Si sceglie il synset con la definizione più simile alla lista delle definizioni, ovvero con più parole in comune

In [55]:
import pandas as pd
import nltk
from collections import Counter
from nltk.corpus import wordnet as wn

### Leggi il file di input

Salvo le varie definizioni degli studenti in un dizionario con:
- chiave: parola (in ordine Emotion, Person, Revenge, Brick)
- valore: lista di definizioni date dagli studenti

In [56]:
df = pd.read_excel('../data/definizioni.xlsx')
students = df.columns[1:].tolist()
words = df[df.columns[0]].tolist()
def_dict = {}
for definition in words:
    def_dict[definition] = []
for student in students:
    for definition in words:
        def_dict[definition].append(df[student][df[df.columns[0]] == definition].values[0])

# def_dict

### Ripulisci il dictionary delle definizioni

Salvo le definizioni pulite in un dizionario strutturato come il precedente

In [57]:
stopwords = nltk.corpus.stopwords.words('english')
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
lemmatizer = nltk.stem.WordNetLemmatizer()

# Tokenize and lemmatize definitions and remove stopwords and punctuation
def clean_definitions(dict):
    cleaned = dict.copy()
    for key in cleaned:
        definitions = [d for d in cleaned[key] if d == d]
        tmp = []
        for definition in definitions:
            tmp.append([lemmatizer.lemmatize(token.lower()) for token in tokenizer.tokenize(definition) if token.lower() not in stopwords])
        cleaned[key] = tmp
    return cleaned

clean_def_dict = clean_definitions(def_dict)

### Trova i genus

Per ogni parola, calcolo le *n* parole più frequenti nelle definizioni date dagli studenti, usando la funzione `most_common` di `Counter`.  
Queste saranno considerate i **genus**.

In [91]:
# Get the most common words in the definitions of a word
def get_common_words(dict, word, n):
    words = []
    for definition in dict[word]:
        words += definition
    return Counter(words).most_common(n)

for word in clean_def_dict:
    print(word, get_common_words(clean_def_dict, word, 5))

Emotion [('feeling', 13), ('human', 8), ('feel', 8), ('something', 7), ('state', 5)]
Person [('human', 29), ('person', 6), ('living', 4), ('individual', 3), ('certain', 3)]
Revenge [('someone', 14), ('anger', 8), ('feeling', 7), ('action', 6), ('emotion', 6)]
Brick [('used', 24), ('object', 16), ('material', 16), ('construction', 16), ('build', 13)]


### Trova tutti gli iponimi di ogni genus

Per ogni genus, trovo tutti gli iponimi e salvo i loro nomi, definizioni e esempi in una lista ripulita.

In [129]:
# Find all hyponyms for all synsets of a word
def find_hyponyms(word):
    synsets = wn.synsets(word)
    hyponyms = []
    for synset in synsets:
        # hyponyms += synset.hyponyms()
        # Closure is used to get all hyponyms for all direct hyponyms
        hyponyms += synset.closure(lambda s:s.hyponyms())
    return set(hyponyms)

# Save hyponyms name, definition and examples in a dictionary
def get_hyponyms_dict(word):
    hyponyms = find_hyponyms(word)
    hyponyms_dict = {}
    for hyponym in hyponyms:
        defs = hyponym.definition()
        examples = hyponym.examples()
        tmp = []
        tmp.append(defs)
        for example in examples:
            tmp.append(example)
        hyponyms_dict[hyponym.name()] = tmp

    return hyponyms_dict

# Preprocess hyponyms definition and examples
def clean_hyponyms_dict(dict):
    cleaned = dict.copy()
    for key in cleaned:
        for definition in cleaned[key]:
            cleaned[key] = [lemmatizer.lemmatize(token.lower()) for token in tokenizer.tokenize(definition) if token.lower() not in stopwords]
    return cleaned

hypo = get_hyponyms_dict('human')
cleaned = clean_hyponyms_dict(hypo)
# print(hypo)
print(cleaned)

{'java_man.n.01': ['fossil', 'remains', 'found', 'java', 'formerly', 'called', 'pithecanthropus', 'erectus'], 'boskop_man.n.01': ['possible', 'early', 'homo', 'sapiens', 'represented', 'cranium', 'found', 'transvaal', 'formerly', 'considered', 'separate', 'specie'], 'neandertal_man.n.01': ['extinct', 'robust', 'human', 'middle', 'paleolithic', 'europe', 'western', 'asia'], 'homo_erectus.n.01': ['homo', 'erectus', 'formerly', 'called', 'pithecanthropus', 'erectus'], 'homo_sapiens_sapiens.n.01': ['subspecies', 'homo', 'sapiens', 'includes', 'modern', 'race'], 'world.n.08': ['always', 'used', 'humankind', 'mankind', 'seemed', 'slight', 'woman'], 'rhodesian_man.n.01': ['primitive', 'hominid', 'resembling', 'neanderthal', 'man', 'living', 'africa'], 'peking_man.n.01': ['fossil', 'found', 'near', 'beijing', 'china', 'lost', 'world', 'war', 'ii'], 'solo_man.n.01': ['early', 'man', 'late', 'pleistocene', 'skull', 'resembles', 'neanderthal', 'man', 'smaller', 'cranial', 'capacity', 'found', 'ja

### Trova il synset più simile alle definizioni

Per ogni iponimo, calcolo la similarità tra la lista delle definizioni date dagli studenti e la definizione dell'iponimo.  
La similarità è calcolata come il numero di parole in comune tra le due liste.

In [133]:
# Compute the similarity between the list of definitions of a word and the definition of a hyponym as the number of common words
def compute_similarity(def_dict, word, hyponym, cleaned_hypo_dict):
    similarity = 0
    for definition in def_dict[word]:
        for word in definition:
            if word in cleaned_hypo_dict[hyponym]:
                # print(f'Definition: {definition}')
                # print(f'Hyponym name: {hyponym}')
                # print(f'Common word: {word}')
                similarity += 1
    return similarity

# Find the most similar hyponym computing the similarity for all hyponyms
def find_most_similar_hypo(def_dict, word, cleaned_hypo_dict):
    most_similar = ''
    max_similarity = 0
    for hypo in cleaned_hypo_dict:
        similarity = compute_similarity(def_dict, word, hypo, cleaned_hypo_dict)
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar = hypo
    # most_similar = most_similar.split('.')[0]
    return most_similar, max_similarity

# Get all hyponyms dicts for a list of words
def hyponyms_list(list):
    hyponyms = []
    for word in list:
        hyponyms += get_hyponyms_dict(word)
    return hyponyms

# Clean a list of hypo dicts
def clean_hyponyms_list(list):
    cleaned = list.copy()
    for i in range(len(cleaned)):
        cleaned[i] = clean_hyponyms_dict(cleaned[i])
    return cleaned

# Find the most similar hyponym for all words
# def find_most_similar_hyponyms(dict):
#     most_similar = {}
#     for word in dict:
#         most_similar[word] = find_most_similar_syn(dict, word)
#     return most_similar


# for word in clean_def_dict:
#     freq_words = get_common_words(clean_def_dict, word, 2)
#     print(word, freq_words)
#     for freq_word in freq_words:
#         print(freq_word[0], find_most_similar_hyponym(clean_def_dict, freq_word[0]))
word = 'Emotion'
freq_words = get_common_words(clean_def_dict, word, 4)
freqs = []
for freq in freq_words: 
    freqs.append(freq[0])
print(freqs)
hypos = hyponyms_list(freqs)
print(hypos)
hypo_cleaned = clean_hyponyms_list(hypos)
print(hypo_cleaned)

# hypo_cleaned = clean_hyponyms_dict(hypos)
# print(hypo_cleaned)
# for hypo in hypo_cleaned:
#     print(compute_similarity(clean_def_dict, word, hypo, hypo_cleaned))
# print(find_most_similar_hypo(clean_def_dict, word, hypo_cleaned))

['feeling', 'human', 'feel', 'something']
['resentment.n.01', 'conflict.n.02', 'addiction.n.02', 'pensiveness.n.01', 'smugness.n.01', 'pruritus_ani.n.01', 'exhilaration.n.01', 'anaphrodisia.n.01', 'pet.n.03', 'anglophobia.n.01', 'eagerness.n.01', 'self-disgust.n.01', 'misoneism.n.01', 'languor.n.01', 'anxiousness.n.02', 'insecurity.n.02', 'loyalty.n.02', 'disgust.n.01', 'timidity.n.01', 'displeasure.n.01', 'ambition.n.01', 'warpath.n.01', 'stupefaction.n.01', 'unassertiveness.n.01', 'pride.n.02', 'agony.n.01', 'happiness.n.01', 'first_blush.n.01', 'steam.v.04', 'anguish.v.01', 'exuberance.n.01', 'warmheartedness.n.01', 'hysteria.n.02', 'misocainea.n.01', 'emotionlessness.n.01', 'stir.n.02', 'testiness.n.01', 'radiance.n.03', 'belligerence.n.01', 'cheer.v.04', 'surprise.n.01', 'compatibility.n.01', 'self-pity.n.01', 'heaviness.n.02', 'worship.n.02', 'schadenfreude.n.01', 'peeve.n.01', 'hesitance.n.01', 'ill_humor.n.01', 'horror.n.01', 'fondness.n.01', 'longing.n.01', 'thing.n.11', 'topo

AttributeError: 'str' object has no attribute 'copy'